In [20]:
import pandas as pd
import numpy as np
import pandas as pd
import os
import re
from sklearn.preprocessing import StandardScaler   
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
# read the dataset from the link, extract and make a dataframe
import requests, zipfile, io
r = requests.get('https://linqs-data.soe.ucsc.edu/public/datasets/pubmed-diabetes/pubmed-diabetes.zip', stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

data_dir = os.path.expanduser("pubmed-diabetes/data")

feature_names = ["w_{}".format(ii) for ii in range(500)]
column_names = ["ID"]+["Label"]+["subject"]+ feature_names 
node_data = pd.read_csv(os.path.join(data_dir, "Pubmed-Diabetes.NODE.paper.tab"), sep='\t|\=', header=None, names=column_names)
node_data.head(10)

/var/folders/k8/fv2pzvcn5p77dgds_c16lsl80000gn/T/ipykernel_51598/3906223784.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  node_data = pd.read_csv(os.path.join(data_dir, "Pubmed-Diabetes.NODE.paper.tab"), sep='\t|\=', header=None, names=column_names)


,ID,Label,subject,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_490,w_491,w_492,w_493,w_494,w_495,w_496,w_497,w_498,w_499
0,NODE,paper,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,cat,"1,2,3:label",numeric:w-rat:0.0,numeric:w-common:0.0,numeric:w-use:0.0,numeric:w-examin:0.0,numeric:w-pathogenesi:0.0,numeric:w-retinopathi:0.0,numeric:w-mous:0.0,numeric:w-studi:0.0,...,numeric:w-urinari:0.0,numeric:w-myocardi:0.0,numeric:w-meal:0.0,numeric:w-ica:0.0,numeric:w-locus:0.0,numeric:w-tcell:0.0,numeric:w-depress:0.0,numeric:w-bone:0.0,numeric:w-mutat:0.0,string:summary
2,12187484,label,1,w-rat,0.09393489570187145,w-common,0.028698458467273157,w-use,0.01176012652514843,w-examin,...,None,None,None,None,None,None,None,None,None,None
3,2344352,label,1,w-rat,0.023617916633613394,w-use,0.014784159060186597,w-anim,0.030926189614601272,w-metabol,...,None,None,None,None,None,None,None,None,None,None
4,14654069,label,1,w-rat,0.10226314418677966,w-use,0.010668980765083111,w-anim,0.04463573758808431,w-contribut,...,None,None,None,None,None,None,None,None,None,None
5,16443886,label,2,w-model,0.038714646134547365,w-develop,0.014074824697725849,w-month,0.023996179817274765,w-method,...,None,None,None,None,None,None,None,None,None,None
6,2684155,label,1,w-rat,0.030615817858387732,w-anim,0.08017901011192922,w-compar,0.019287814631166137,w-normal,...,None,None,None,None,None,None,None,None,None,None
7,15032912,label,1,w-rat,0.11689675909566226,w-studi,0.005201366066481255,w-metabol,0.018465876498282723,w-2,...,None,None,None,None,None,None,None,None,None,None
8,17988185,label,3,w-use,0.007445259958367352,w-studi,0.01111371022838081,w-model,0.013229825117920863,w-2,...,None,None,None,None,None,None,None,None,None,None
9,9834350,label,3,w-2,0.006157469794431223,w-compar,0.0318839384719277,w-6,0.026709793561887893,w-level,...,None,None,None,None,None,None,None,None,None,None


In [2]:
import numpy as np
DataS = node_data.drop(labels=0, axis=0)
DataS = DataS.fillna(value=np.nan)
DataS.head()

,ID,Label,subject,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_490,w_491,w_492,w_493,w_494,w_495,w_496,w_497,w_498,w_499
1,cat,"1,2,3:label",numeric:w-rat:0.0,numeric:w-common:0.0,numeric:w-use:0.0,numeric:w-examin:0.0,numeric:w-pathogenesi:0.0,numeric:w-retinopathi:0.0,numeric:w-mous:0.0,numeric:w-studi:0.0,...,numeric:w-urinari:0.0,numeric:w-myocardi:0.0,numeric:w-meal:0.0,numeric:w-ica:0.0,numeric:w-locus:0.0,numeric:w-tcell:0.0,numeric:w-depress:0.0,numeric:w-bone:0.0,numeric:w-mutat:0.0,string:summary
2,12187484,label,1,w-rat,0.09393489570187145,w-common,0.028698458467273157,w-use,0.01176012652514843,w-examin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2344352,label,1,w-rat,0.023617916633613394,w-use,0.014784159060186597,w-anim,0.030926189614601272,w-metabol,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14654069,label,1,w-rat,0.10226314418677966,w-use,0.010668980765083111,w-anim,0.04463573758808431,w-contribut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,16443886,label,2,w-model,0.038714646134547365,w-develop,0.014074824697725849,w-month,0.023996179817274765,w-method,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
DataS=DataS.replace({'subject': {'1':0 , '2': 1,'3':2}})
# label 1=0 
#		label 2= 1
#		label 3=2


DataS.head()

,ID,Label,subject,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_490,w_491,w_492,w_493,w_494,w_495,w_496,w_497,w_498,w_499
1,cat,"1,2,3:label",numeric:w-rat:0.0,numeric:w-common:0.0,numeric:w-use:0.0,numeric:w-examin:0.0,numeric:w-pathogenesi:0.0,numeric:w-retinopathi:0.0,numeric:w-mous:0.0,numeric:w-studi:0.0,...,numeric:w-urinari:0.0,numeric:w-myocardi:0.0,numeric:w-meal:0.0,numeric:w-ica:0.0,numeric:w-locus:0.0,numeric:w-tcell:0.0,numeric:w-depress:0.0,numeric:w-bone:0.0,numeric:w-mutat:0.0,string:summary
2,12187484,label,0,w-rat,0.09393489570187145,w-common,0.028698458467273157,w-use,0.01176012652514843,w-examin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2344352,label,0,w-rat,0.023617916633613394,w-use,0.014784159060186597,w-anim,0.030926189614601272,w-metabol,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14654069,label,0,w-rat,0.10226314418677966,w-use,0.010668980765083111,w-anim,0.04463573758808431,w-contribut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,16443886,label,1,w-model,0.038714646134547365,w-develop,0.014074824697725849,w-month,0.023996179817274765,w-method,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from natsort import index_natsorted, order_by_index

Data = DataS.reindex(index=order_by_index(DataS.index, index_natsorted(DataS['ID'], reverse=False)))
Data.head()

,ID,Label,subject,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_490,w_491,w_492,w_493,w_494,w_495,w_496,w_497,w_498,w_499
12589,7145,label,0,w-rat,0.14376123168286414,w-examin,0.024710673888640275,w-studi,0.007462829573647018,w-metabol,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19549,29094,label,0,w-rat,0.09538004794343873,w-use,0.01990175258102042,w-compar,0.020029653655441758,w-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13218,34420,label,0,w-rat,0.11323658659951627,w-anim,0.05931050063074217,w-2,0.008266192326770682,w-obtain,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10331,34548,label,0,w-studi,0.007356217722594917,w-anim,0.030926189614601272,w-mice,0.1523641186469851,w-increas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13973,37920,label,0,w-rat,0.1377711803627448,w-studi,0.01430375668282345,w-anim,0.060134257583946915,w-investig,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
Data.tail()

,ID,Label,subject,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_490,w_491,w_492,w_493,w_494,w_495,w_496,w_497,w_498,w_499
3426,20003208,label,2,w-studi,0.005536938070770368,w-2,0.006488516557572687,w-month,0.04902445339013124,w-compar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4427,20011163,label,1,w-use,0.012175189814271315,w-studi,0.012116123307803393,w-anim,0.05093725348287269,w-model,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11940,20061358,label,2,w-studi,0.009855219915438166,w-2,0.01443617320225502,w-compar,0.014950363494013944,w-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5857,20061360,label,2,w-use,0.004835939879500289,w-studi,0.0024062394419703,w-investig,0.008284388292780094,w-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cat,"1,2,3:label",numeric:w-rat:0.0,numeric:w-common:0.0,numeric:w-use:0.0,numeric:w-examin:0.0,numeric:w-pathogenesi:0.0,numeric:w-retinopathi:0.0,numeric:w-mous:0.0,numeric:w-studi:0.0,...,numeric:w-urinari:0.0,numeric:w-myocardi:0.0,numeric:w-meal:0.0,numeric:w-ica:0.0,numeric:w-locus:0.0,numeric:w-tcell:0.0,numeric:w-depress:0.0,numeric:w-bone:0.0,numeric:w-mutat:0.0,string:summary


In [6]:
f=node_data.loc[1, feature_names].values.flatten().tolist()
f.insert(0,'numeric:w-rat:0.0')
f.remove('string:summary')
#print(f)
word=[]
for x in f:
    x=x.replace("numeric:","")
    x=x.replace(":0.0","")
    word.append(x)
#print(word)    
print(len(word))

500


In [7]:
Sub=Data['subject'].to_numpy()
Sub=np.delete(Sub,-1)
print(len(Sub))

19717


In [8]:
IdS=list(DataS['ID'].to_numpy())
Id=list(Data['ID'].to_numpy())
Index=[]
for s in Id:
    Index.append(IdS.index(s)+1)
print(len(Index))
Fec=[]
for i in range(19717):
    #print("\rProcessing file {} ({}%)".format(i, 100*i//(19717)), end='', flush=True)
    vec=[]
    f=Data.loc[Index[i], feature_names].values.flatten().tolist()
    
    target_ibdex = f.index('summary')
    d=f[:target_ibdex]
    for z in word:
        if z in d:
            vec.append(d[d.index(z)+1])
        else:
            vec.append(0)
    f.clear()
    Fec.append(vec)
#print(Fec)
df = pd.DataFrame(Fec, columns =word) 
#df.insert(loc=500,column='subject',value=Data['subject'].to_numpy())
df.head()

19718


,w-rat,w-common,w-use,w-examin,w-pathogenesi,w-retinopathi,w-mous,w-studi,w-anim,w-model,...,w-kidney,w-urinari,w-myocardi,w-meal,w-ica,w-locus,w-tcell,w-depress,w-bone,w-mutat
0,0.14376123168286414,0,0,0.024710673888640275,0,0,0,0.007462829573647018,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.09538004794343873,0,0.01990175258102042,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.11323658659951627,0,0,0,0,0,0,0,0.05931050063074217,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0.007356217722594917,0.030926189614601272,0,...,0,0,0,0,0,0,0,0,0,0
4,0.1377711803627448,0,0,0,0,0,0,0.01430375668282345,0.060134257583946915,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# import all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [10]:
# Scale data before applying PCA
scaling=StandardScaler()
 
# Use fit and transform method
scaling.fit(df)
Scaled_data=scaling.transform(df)
 
# Set the n_components=3
m=100
principal=PCA(n_components=m)
principal.fit(Scaled_data)
x=principal.transform(Scaled_data)
 
# Check the dimensions of data after PCA
print(x.shape)

(19717, 100)


In [11]:
feature_names=[]
for i in range(m):
    feature_names.append("P_{}".format(i))
Data1 = pd.DataFrame(x,columns=feature_names)
Data1.head()

,P_0,P_1,P_2,P_3,P_4,P_5,P_6,P_7,P_8,P_9,...,P_90,P_91,P_92,P_93,P_94,P_95,P_96,P_97,P_98,P_99
0,2.557717,-4.878935,0.152017,-3.212815,0.976183,0.513569,-2.234455,-0.480674,0.019857,-1.884633,...,0.689582,0.797522,-0.293991,0.359047,-1.155146,-0.748994,-1.961687,1.096754,0.522165,-0.310279
1,1.679335,-4.901470,0.563493,-1.929009,0.004088,-0.867592,-0.832645,1.459545,0.124936,-1.029924,...,-1.399305,-0.809555,-0.097219,-2.153605,1.037081,-1.289312,-2.101206,-1.543410,0.069899,0.851963
2,1.752778,-3.757600,1.535269,-2.391571,-1.648668,-0.422477,-0.719876,2.327269,-3.074637,3.761228,...,-1.950914,-0.268680,-2.155114,2.885071,0.993921,-1.971582,0.221032,-0.760566,1.063734,0.951995
3,-0.789080,-3.760483,-2.439622,-0.503694,0.286759,0.283389,-0.600328,-0.722383,-0.817884,-0.758927,...,-1.065378,-0.771763,-1.092592,-0.285575,-0.384110,1.513792,-0.835163,-2.562681,-0.307905,1.034544
4,3.133193,-4.369138,0.755052,-2.670575,1.316855,-1.779786,-0.435492,1.192098,-1.093630,-0.189683,...,0.526435,0.193368,-1.597801,2.192633,1.420331,-0.857822,2.230776,0.160108,-2.557070,-0.618395


In [12]:
p_data = open("pubmed-diabetes/data/Pubmed-Diabetes.DIRECTED.cites.tab")


# Create a list from the data
myls=list(p_data)

# Remove the new line symbole from the list
mylist = [line.rstrip('\n') for line in myls]
mylist=np.delete(mylist,[0,1])
#print(graph_ind)
#print(graph_level)

p_data.close()
mynode=[]
Node=[]
i=0
for d in mylist:
    d=mylist[i].split("\t")
    mynode.append(d[1])
    Node.append(d[0])
    mynode.append(d[3])
    i=i+1
#print(mynode)
node_ID1=np.unique(mynode)
node_ID2=list(node_ID1)
#print(node_ID2)
n=len(node_ID2)

In [13]:
from natsort import natsorted, ns
node_ID=natsorted(node_ID2)
#print(node_ID)

A=np.zeros((n,n))
i=0
for d in mylist:
    d=mylist[i].split("\t")
    A[node_ID.index(d[1])][node_ID.index(d[3])]=1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

catagories=Sub
print(catagories)


 The adjacency Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0 0 0 ... 1 2 2]


In [14]:
import random
p=random.sample(range(19717), 1000)
#print(p)
q=[]
for i in range(n):
    if i not in p:
        q.append(i)

class_60=np.array(catagories)
for d in p:
    class_60[d]=3
print(class_60)
for i in range(n):
    A[i][i]=class_60[i]
print(A)

[0 0 0 ... 1 2 3]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 2. 0.]
 [0. 0. 0. ... 0. 0. 3.]]


In [15]:
def CountX(lst,x):
    return(lst.count(x))

def Spatial_two(Node_class,A,n):
    F_vec=[]
    for i in range(n):
        print("\rProcessing file {} ({}%)".format(i, 100*i//(n-1)), end='', flush=True)
        node_F=[]
        list_out=[]
        list_In=[]
        S_nbd_out=[]
        S_nbd_in=[]
        for j in range(n):
            if A[i][j]==1 and i!=j:
                list_out.append(A[j][j])
                for k in range(n):
                    if A[j][k]==1 and k!=j:
                        S_nbd_out.append(A[k][k])
            if A[j][i]==1 and i!=j:
                list_In.append(A[j][j])
                for k in range(n):
                    if A[k][j]==1 and k!=j:
                        S_nbd_in.append(A[k][k])

        for d in Node_class:

            node_F.append(CountX(list_out,d))
            node_F.append(CountX(list_In,d))
        for d in Node_class:
            node_F.append(CountX(S_nbd_out,d))
            node_F.append(CountX(S_nbd_in,d))
        F_vec.append(node_F)
    return F_vec

In [27]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

In [30]:
import random
for it in range(2):
    p=[]
    q=[]
    p=random.sample(range(19717), 1000)
    #print(p)
    q=[]
    for i in range(n):
        if i not in p:
            q.append(i)

    class_60=np.array(catagories)
    for d in p:
        class_60[d]=3
    #print(class_60)
    for i in range(n):
        A[i][i]=class_60[i]

    Node_class=[0,1,2]
    F_vec=Spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=len(F_vec[0])
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Sub)
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    y=y.astype('int') 
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

 
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)

    
    Ac_domain2.append(metrics.accuracy_score(y_test, y_pred)*100)
    result = pd.concat([Data1,data], axis=1)
    k=len(F_vec[0])
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    l=100
    for i in range(l):
        feature.append("P_{}".format(i))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    y=y.astype('int') 
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    acuracy.append(metrics.accuracy_score(y_test, y_pred)*100)
    feature=[]
    l=100
    for i in range(l):
        feature.append("P_{}".format(i))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    y=y.astype('int') 
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    Ac_fe.append(metrics.accuracy_score(y_test, y_pred)*100)

Processing file 19716 (100%)

In [29]:
import statistics
mean_value = statistics.mean(Ac_domain2)
std_dev = statistics.stdev(Ac_domain2)
print("Spatial Only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(Ac_fe)
std_dev = statistics.stdev(Ac_fe)
print("Domain only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(acuracy)
std_dev = statistics.stdev(acuracy)
print("Accuracy using both")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

Spatial Only
Mean: 81.55
Standard Deviation: 1.6263455967290472
Domain only
Mean: 82.6
Standard Deviation: 0.14142135623730145
Accuracy using both
Mean: 87.15
Standard Deviation: 1.0606601717798212


In [31]:
import statistics
mean_value = statistics.mean(Ac_domain2)
std_dev = statistics.stdev(Ac_domain2)
print("Spatial Only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(Ac_fe)
std_dev = statistics.stdev(Ac_fe)
print("Domain only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(acuracy)
std_dev = statistics.stdev(acuracy)
print("Accuracy using both")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

Spatial Only
Mean: 81.89999999999999
Standard Deviation: 1.1165422816296098
Domain only
Mean: 83.89999999999999
Standard Deviation: 1.5383974345619105
Accuracy using both
Mean: 88.075
Standard Deviation: 1.325078614020065


In [164]:
x =np.array(F_vec)
k=16

feature=["a_0","b_0"]
for i in range(1,k-1,2):
    feature.append("a_{}".format(int((i+1)/2)))
    feature.append("b_{}".format(int((i+1)/2)))
data = pd.DataFrame(x,columns=feature)
data.insert(loc=k,column='Class',value=Sub)
data.head()

,a_0,b_0,a_1,b_1,a_2,b_2,a_3,b_3,a_4,b_4,a_5,b_5,a_6,b_6,a_7,b_7,Class
0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,4,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,5,0,1,0,0,0,0,0
4,0,1,0,1,0,0,0,0,0,0,0,3,0,0,0,0,0


In [165]:
result = pd.concat([Data1,data], axis=1)
result.head()

,P_0,P_1,P_2,P_3,P_4,P_5,P_6,P_7,P_8,P_9,...,b_3,a_4,b_4,a_5,b_5,a_6,b_6,a_7,b_7,Class
0,2.557737,-4.878860,0.152612,-3.213854,0.974697,0.504368,-2.232107,-0.473480,0.011093,-1.903368,...,0,0,0,0,0,0,0,0,0,0
1,1.679341,-4.901389,0.563107,-1.929921,0.010230,-0.878932,-0.826590,1.448566,0.121083,-1.039214,...,0,0,4,0,0,0,0,0,0,0
2,1.752810,-3.757506,1.536405,-2.389533,-1.665371,-0.405541,-0.704739,2.351341,-3.085322,3.739644,...,0,0,1,0,0,0,0,0,0,0
3,-0.789096,-3.760393,-2.440032,-0.502391,0.280188,0.288367,-0.599763,-0.721824,-0.817742,-0.753638,...,0,0,5,0,1,0,0,0,0,0
4,3.133174,-4.369068,0.756366,-2.671206,1.318050,-1.777679,-0.441984,1.191812,-1.127028,-0.168845,...,0,0,0,0,3,0,0,0,0,0


In [148]:
k=16
feature=["a_0","b_0"]
for i in range(1,k-1,2):
    feature.append("a_{}".format(int((i+1)/2)))
    feature.append("b_{}".format(int((i+1)/2)))
l=50
for i in range(l):
    feature.append("P_{}".format(i))

    
    
X=result[feature] # Features
y=result['Class']  # Labels
y=y.astype('int') 
X_train=X.iloc[q]
X_test=X.iloc[p]
y_train=y.iloc[q]
y_test=y.iloc[p]

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)  

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")
Both.append(metrics.accuracy_score(y_test, y_pred)*100)

Accuracy: 86.6 %


In [29]:
D_only=[]
S_only=[]
Both=[]

In [30]:
Both.append(metrics.accuracy_score(y_test, y_pred)*100)

In [60]:
k=16
feature=["a_0","b_0"]
for i in range(1,k-1,2):
    feature.append("a_{}".format(int((i+1)/2)))
    feature.append("b_{}".format(int((i+1)/2)))
#l=50
#for i in range(l):
#    feature.append("P_{}".format(i))

    
    
X=result[feature] # Features
y=result['Class']  # Labels
y=y.astype('int')
X_train=X.iloc[q]
X_test=X.iloc[p]
y_train=y.iloc[q]
y_test=y.iloc[p]

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)  

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")
S_only.append(metrics.accuracy_score(y_test, y_pred)*100)

Accuracy: 83.3 %


In [61]:
#k=16
#feature=["a_0","b_0"]
#for i in range(1,k-1,2):
#    feature.append("a_{}".format(int((i+1)/2)))
#    feature.append("b_{}".format(int((i+1)/2)))
l=100
for i in range(l):
    feature.append("P_{}".format(i))

    
    
X=result[feature] # Features
y=result['Class']  # Labels
y=y.astype('int')
X_train=X.iloc[q]
X_test=X.iloc[p]
y_train=y.iloc[q]
y_test=y.iloc[p]

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)  

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")
D_only.append(metrics.accuracy_score(y_test, y_pred)*100)

Accuracy: 93.10000000000001 %


In [67]:
print(statistics.mean(Both))
print(statistics.stdev(Both))
print(statistics.mean(S_only))
print(statistics.stdev(S_only))
print(statistics.mean(D_only))
print(statistics.stdev(D_only))

90.425
0.6344288770224753
81.575
1.1842719282326968
91.72500000000001
1.6700798384109317
